In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt      #import knižníc
import re
import datetime
import matplotlib.dates as mdates
import seaborn as sns
from datetime import timedelta

# Načítanie počiatočných dát

In [3]:
goes_xray_event_list_2013 = pd.read_csv("goes_xray_event_list_2013.csv")      #načítanie datasetov
full_df = pd.read_csv("full_df.csv")
full_df['time'] = pd.to_datetime(full_df['time'])

In [2]:
file_path = "goes_xray_event_list_2013.txt"
df = pd.read_fwf(file_path, header=None)

def check_conditions(value):
    if value.startswith('M') or value.startswith('X'):
        return True
    elif value.startswith('C'):            #výber udalostí začínajúcich na M,X, alebo C od 5.0 vyššie
        number_part = float(value[1:])
        return number_part >= 5.0
    return False

filtered_df = df[df[4].apply(check_conditions)]
goes_xray_event_list_2013 = filtered_df.iloc[:, :5] #premenovanie stĺpcov
goes_xray_event_list_2013.rename(columns={0: 'Date', 1: 'Start', 2: 'Peak', 3: 'End', 4: 'Label'}, inplace=True)

goes_xray_event_list_2013['Date'] = pd.to_datetime(goes_xray_event_list_2013['Date'], format='%d-%b-%Y') #zmena formátu dátumu

def combine_date_time(date, time):
    datetime_str = date.strftime('%Y-%m-%d') + ' ' + time
    return pd.to_datetime(datetime_str, format='%Y-%m-%d %H:%M')    #zmena formátu dátumu

for col in ['Start', 'Peak', 'End']:
    goes_xray_event_list_2013[col] = goes_xray_event_list_2013.apply(lambda row: combine_date_time(row['Date'], row[col]), axis=1)
       
goes_xray_event_list_2013 = goes_xray_event_list_2013[
    (goes_xray_event_list_2013['Start'].dt.time >= pd.to_datetime('09:00').time()) & #výber udalostí medzi 09:00-17:00
    (goes_xray_event_list_2013['End'].dt.time <= pd.to_datetime('17:00').time())
]
goes_xray_event_list_2013.to_csv('goes_xray_event_list_2013.csv', index=False) #uloženie datasetu

In [32]:
goes_xray_event_list_2013.to_csv('goes_xray_event_list_2013.csv', index=False) #uloženie datasetu

In [3]:
def format_time(time_str):
    parts = time_str.split(":")
    while len(parts) < 3:
        parts.append('00')
    formatted_time = ":".join(parts) #zmena formátu času
    return formatted_time

folder_path = "2013_z"
file_list = os.listdir(folder_path)   #načítanie zložky obsahujúcej dáta
data_frames = {}
last_time_previous_file = None

for file in file_list:
    if file.endswith(".BEL"): 
        file_path = os.path.join(folder_path, file)          #výber súborov podľa typu súboru
        df_name = os.path.splitext(file)[0]
        with open(file_path) as f:
            lines = f.readlines()
        lst = []
        for line in lines:
            if line.startswith(' '):
                line = line.replace(' ', '0', 1)  
            if line.startswith('0') or line.startswith('1') or line.startswith('2'): #výber riadkov začínajúcich na 0,1, alebo 2
                parts = line.split()
                lst.append(parts[0:])
        df = pd.DataFrame(lst)
        day = int(df_name[1:3])
        month_str = df_name[3] + df_name[4:6].lower()
        year = 2013
        month = datetime.datetime.strptime(month_str, "%b").month #tvorba časovej známky
        base_date = datetime.date(year, month, day)

        df[0] = df[0].apply(format_time)
        df[0] = pd.to_datetime(df[0].apply(lambda x: base_date.strftime("%Y-%m-%d") + ' ' + x))

        for i in range(len(df)):
            current_time = df.at[i, 0].time()
            if i > 0 and current_time < df.at[i - 1, 0].time():
                base_date += datetime.timedelta(days=1)
            df.at[i, 0] = datetime.datetime.combine(base_date, current_time) #ošetrenie prechodu do druhého dňa

        last_time_previous_file = df.at[len(df) - 1, 0]

        df = df.drop(columns=[1, 2, 4, 6, 8, 10, 12])
        df = df.rename(columns={3:'20.27kHz', 5:'23.4kHz', 7:'22.1kHz', 9:'19.8kHz', 11:'24kHz'}) #zmena názvu stĺpcov
        data_frames[df_name] = df

full_df = pd.concat(data_frames.values(), ignore_index=True)

full_df.rename(columns={0: 'time'}, inplace=True) #zmena názvu stĺpcov

full_df = full_df.sort_values(by='time')  #zoradenie podľa časovej známky

In [34]:
full_df

,time,20.27kHz,23.4kHz,22.1kHz,19.8kHz,24kHz,label,month_year
35030,2013-01-06 01:25:00,74.448,24.6165,73.083,44.3502,64.755,0,2013-01
35031,2013-01-06 01:26:00,74.5246,24.3537,73.1161,44.5997,64.6402,0,2013-01
35032,2013-01-06 01:27:00,74.6337,24.3696,73.1212,45.2772,64.6167,0,2013-01
35033,2013-01-06 01:28:00,74.6671,25.1063,73.0995,45.1751,64.6046,0,2013-01
35034,2013-01-06 01:29:00,74.544,25.3732,73.0995,45.0304,64.5481,0,2013-01
...,...,...,...,...,...,...,...,...
251595,2013-12-23 12:20:00,-3.01,-3.01,-3.01,-3.01,-3.01,0,2013-12
251596,2013-12-23 12:21:00,-3.01,-3.01,-3.01,-3.01,-3.01,0,2013-12
251597,2013-12-23 12:22:00,-3.01,-3.01,-3.01,-3.01,-3.01,0,2013-12
251598,2013-12-23 12:23:00,-3.01,-3.01,-3.01,-3.01,-3.01,0,2013-12


In [6]:
full_df['label'] = pd.Series(['0'] * len(full_df), dtype=object)

for i, full_row in full_df.iterrows():
    for _, result_row in goes_xray_event_list_2013.iterrows():
        if full_row['time'].date() == result_row['Date'].date():
            if result_row['Start'] <= full_row['time'] <= result_row['End']: #porovnanie dvoch datasetov podľa časovej známky, priradenie labelu
                full_df.at[i, 'label'] = result_row['Label']
                break

full_df

,time,20.27kHz,23.4kHz,22.1kHz,19.8kHz,24kHz,label
35030,2013-01-06 01:25:00,74.448,24.6165,73.083,44.3502,64.755,0
35031,2013-01-06 01:26:00,74.5246,24.3537,73.1161,44.5997,64.6402,0
35032,2013-01-06 01:27:00,74.6337,24.3696,73.1212,45.2772,64.6167,0
35033,2013-01-06 01:28:00,74.6671,25.1063,73.0995,45.1751,64.6046,0
35034,2013-01-06 01:29:00,74.544,25.3732,73.0995,45.0304,64.5481,0
...,...,...,...,...,...,...,...
251595,2013-12-23 12:20:00,-3.01,-3.01,-3.01,-3.01,-3.01,0
251596,2013-12-23 12:21:00,-3.01,-3.01,-3.01,-3.01,-3.01,0
251597,2013-12-23 12:22:00,-3.01,-3.01,-3.01,-3.01,-3.01,0
251598,2013-12-23 12:23:00,-3.01,-3.01,-3.01,-3.01,-3.01,0


# Filtrovanie iba dát s labelom

In [9]:
filtered_df = full_df[full_df['label']!='0']
filtered_df

,time,20.27kHz,23.4kHz,22.1kHz,19.8kHz,24kHz,label
43036,2013-01-11 14:51:00,68.1021,69.5749,61.7041,46.9264,44.3739,M1.0
43037,2013-01-11 14:52:00,68.1111,69.4487,61.6187,47.7931,44.2178,M1.0
43038,2013-01-11 14:53:00,68.1416,69.3582,61.501,49.5478,44.2159,M1.0
43039,2013-01-11 14:54:00,69.8025,69.1604,61.4183,49.711,44.2315,M1.0
43040,2013-01-11 14:55:00,69.803,68.7607,61.4311,49.752,44.483,M1.0
...,...,...,...,...,...,...,...
233641,2013-12-22 15:15:00,-3.01,-3.01,3.01877,-3.01,-1.50076,M3.3
233642,2013-12-22 15:16:00,-3.01,-3.01,0.00847404,-3.01,-1.50076,M3.3
233643,2013-12-22 15:17:00,-3.01,-3.01,0.00847404,-3.01,-1.50076,M3.3
233644,2013-12-22 15:18:00,-3.01,-3.01,3.01877,-3.01,0.00847404,M3.3


In [10]:
full_df.to_csv('full_df.csv', index=False)

In [ ]:
file_path = 'full_df.csv'
df = pd.read_csv(file_path)  #načítanie datasetu

df['time'] = pd.to_datetime(df['time'])

frequencies = [col for col in df.columns if 'kHz' in col]

def plot_data_for_label(df, label, start_time, end_time, frequency):
    plot_start_time = start_time - timedelta(hours=1)
    plot_end_time = end_time + timedelta(hours=12)
    plot_df = df[(df['time'] >= plot_start_time) & (df['time'] <= plot_end_time)] #hodina pred začiatkom, 12 hodín po začiatku

    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(plot_df['time'], plot_df[frequency], label=frequency) #graf
    
    ax.axvline(x=start_time, color='red', linestyle='--', label='First Occurrence')
    ax.axvline(x=end_time, color='red', linestyle='--', label='Last Occurrence') #označenie udalosti

    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
    plt.xticks(rotation=45)
    plt.xlabel('Time')
    plt.ylabel('Frequency Value')
    plt.title(f'Frequency {frequency} with Label {label}')   #tvorba popiskov
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'uniquePlots/{label}_{frequency.replace("kHz", "")}_plot.png')
    plt.show()

label_groups = df[df['label'] != '0'].groupby('label')

label_occurrences = []

for label, group in label_groups:
    group = group.sort_values('time')

    start_time = None
    end_time = None

    for i, row in group.iterrows():
        if start_time is None:
            start_time = row['time']
            end_time = row['time']
        elif row['time'] - end_time > timedelta(hours=1):
            label_occurrences.append((label, start_time, end_time))
            start_time = row['time']
            end_time = row['time']
        else:
            end_time = row['time']

    label_occurrences.append((label, start_time, end_time))

for frequency in frequencies:
    for label, start, end in label_occurrences:
        plot_data_for_label(df, label, start, end, frequency)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import timedelta

file_path = 'full_df.csv'
df = pd.read_csv(file_path)

df['time'] = pd.to_datetime(df['time'])

frequencies = [col for col in df.columns if 'kHz' in col]

def plot_data_for_label(df, label, start_time, end_time, frequency):
    plot_start_time = start_time - timedelta(hours=1)
    plot_end_time = end_time + timedelta(hours=12)
    plot_df = df[(df['time'] >= plot_start_time) & (df['time'] <= plot_end_time)]  #výber časového okna pre vykreslenie

    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(plot_df['time'], plot_df[frequency], label=frequency)
    
    ax.axvline(x=start_time, color='red', linestyle='--', label='Start of Occurrence') #označenie udalosti v grafe
    ax.axvline(x=end_time, color='red', linestyle='--', label='End of Occurrence')

    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
    plt.xticks(rotation=45)
    plt.xlabel('Time')
    plt.ylabel('Frequency Value')
    plt.title(f'Frequency {frequency} with Label {label}') #tvorba popiskov
    plt.legend()
    plt.tight_layout()
    plot_filename = f'qwerty/{label}_{frequency.replace("kHz", "")}_{start_time.strftime("%Y%m%d%H%M%S")}_plot.png'
    plt.savefig(plot_filename)
    plt.show()

for frequency in frequencies:
    current_label = None
    start_time = None
    end_time = None

    for i, row in df.iterrows():
        if row['label'] != '0':
            if current_label is None:
                current_label = row['label']     #označenie labelu v grafe
                start_time = row['time']
            end_time = row['time']
        else:
            if current_label is not None:
                plot_data_for_label(df, current_label, start_time, end_time, frequency)
                current_label = None
                start_time = None
                end_time = None

    if current_label is not None:
        plot_data_for_label(df, current_label, start_time, end_time, frequency)

# Grafy

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import timedelta

file_path = 'full_df.csv'
df = pd.read_csv(file_path)

df['time'] = pd.to_datetime(df['time'])

frequencies = [col for col in df.columns if 'kHz' in col]

def plot_data_for_label(df, label, start_time, end_time, frequency, plot_index):  #hodina pred udalosťou, 12 hodín po
    plot_start_time = start_time - timedelta(hours=1)
    plot_end_time = end_time + timedelta(hours=12)
    plot_df = df[(df['time'] >= plot_start_time) & (df['time'] <= plot_end_time)]

    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(plot_df['time'], plot_df[frequency], label=frequency)
    
    ax.axvline(x=start_time, color='red', linestyle='--', label='Start of Occurrence')  #línia na vyznačenie udalosti
    ax.axvline(x=end_time, color='red', linestyle='--', label='End of Occurrence')

    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
    plt.xticks(rotation=45)
    plt.xlabel('Time')
    plt.ylabel('Frequency Value')
    plt.title(f'Frequency {frequency} with Label {label}')     #popisky
    plt.legend()
    plt.tight_layout()
    plot_filename = f'qwer/{label}_{frequency.replace("kHz", "")}_{plot_index}_plot.png'
    plt.savefig(plot_filename)
    plt.close()

plot_index = 0
for frequency in frequencies:
    current_label = None
    start_time = None
    end_time = None

    for i, row in df.iterrows():
        if row['label'] != '0':
            if current_label is None:
                current_label = row['label']
                start_time = row['time']
            end_time = row['time']
        else:
            if current_label is not None:
                plot_data_for_label(df, current_label, start_time, end_time, frequency, plot_index) #vykresľovanie
                plot_index += 1
                current_label = None
                start_time = None
                end_time = None

    if current_label is not None:
        plot_data_for_label(df, current_label, start_time, end_time, frequency, plot_index)
        plot_index += 1

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import timedelta

file_path = 'full_df.csv'
df = pd.read_csv(file_path)

df['time'] = pd.to_datetime(df['time'])

frequencies = [col for col in df.columns if 'kHz' in col]

def plot_data_for_label(df, label, start_time, end_time, frequency, plot_index):
    plot_start_time = start_time - timedelta(hours=1)
    plot_end_time = end_time + timedelta(hours=12)
    plot_df = df[(df['time'] >= plot_start_time) & (df['time'] <= plot_end_time)]

    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(plot_df['time'], plot_df[frequency], label=frequency)
    
    ax.axvline(x=start_time, color='red', linestyle='--', label='Start of Occurrence')
    ax.axvline(x=end_time, color='red', linestyle='--', label='End of Occurrence')

    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
    plt.xticks(rotation=45)
    plt.xlabel('Time')
    plt.ylabel('Frequency Value')
    plt.title(f'Frequency {frequency} with Label {label}')
    plt.legend()
    plt.tight_layout()
    plot_filename = f'uniquePlots/{label}_{frequency.replace("kHz", "")}_{plot_index}_plot.png'
    plt.savefig(plot_filename)
    plt.close()

plot_index = 0
for frequency in frequencies:
    current_label = None
    start_time = None
    end_time = None

    for i, row in df.iterrows():
        if row['label'] != '0':
            if current_label is None:
                current_label = row['label']
                start_time = row['time']
            end_time = row['time']
        else:
            if current_label is not None:
                plot_data_for_label(df, current_label, start_time, end_time, frequency, plot_index) #vykresľovanie
                plot_index += 1
                current_label = None
                start_time = None
                end_time = None

    if current_label is not None:
        plot_data_for_label(df, current_label, start_time, end_time, frequency, plot_index)
        plot_index += 1

# Dáta zo Srbska

In [3]:
goes_xray_event_list_2013_Serbia = pd.read_csv('goes_xray_event_list_2013_Serbia.csv') #načítanie datasetu

In [4]:
goes_xray_event_list_2013_Serbia

,Date,Start,Peak,End,Label,19.8,20.27,22.1,23.4,24,Unnamed: 10,Unnamed: 11
0,2013-10-18,2013-10-18 10:17:00,2013-10-18 10:34:00,2013-10-18 10:48:00,C5.0,-,+,-,+,x,NaN,NaN
1,2013-12-29,2013-12-29 14:39:00,2013-12-29 14:46:00,2013-12-29 14:50:00,C5.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-10-28,2013-10-28 09:15:00,2013-10-28 09:22:00,2013-10-28 09:26:00,C5.2,-,-,-,-,-,NaN,NaN
3,2013-05-13,2013-05-13 13:55:00,2013-05-13 14:40:00,2013-05-13 15:10:00,C5.3,-,x,x,x,-,more flares,NaN
4,2013-12-23,2013-12-23 15:49:00,2013-12-23 15:53:00,2013-12-23 15:56:00,C5.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
71,2013-10-28,2013-10-28 15:07:00,2013-10-28 15:15:00,2013-10-28 15:21:00,M4.4,-,-,-,-,-,NaN,NaN
72,2013-05-22,2013-05-22 13:08:00,2013-05-22 13:32:00,2013-05-22 14:08:00,M5.0,NaN,+?,+ D+I ?,+ ?,+ ?,it seems that flare start earlier – check if o...,NaN
73,2013-11-19,2013-11-19 10:14:00,2013-11-19 10:26:00,2013-11-19 10:34:00,X1.0,+,+,+,+,-,NaN,NaN
74,2013-10-25,2013-10-25 14:51:00,2013-10-25 15:03:00,2013-10-25 15:12:00,X2.1,-,-,-,-,-,NaN,NaN


In [5]:
full_df = pd.read_csv('full_df.csv')

In [6]:
full_df['time'] = pd.to_datetime(full_df['time'])

In [7]:
khz2027 = full_df.loc[:,['time','20.27kHz','label']]
khz234 = full_df.loc[:,['time','23.4kHz','label']]
khz221 = full_df.loc[:,['time','22.1kHz','label']] #tvorba datasetov pre každú frekvenciu
khz198 = full_df.loc[:,['time','19.8kHz','label']]
khz24 = full_df.loc[:,['time','24kHz','label']]

In [8]:
khz2027['label'] = 0
khz234['label'] = 0
khz221['label'] = 0
khz198['label'] = 0
khz24['label'] = 0
khz2027['time'] = pd.to_datetime(khz2027['time']) #formátovanie
khz234['time'] = pd.to_datetime(khz234['time'])
khz221['time'] = pd.to_datetime(khz221['time'])
khz198['time'] = pd.to_datetime(khz198['time'])
khz24['time'] = pd.to_datetime(khz24['time'])

In [9]:
goes_df = pd.read_csv('goes_xray_event_list_2013_Serbia.csv')

khz198['time'] = pd.to_datetime(khz198['time'])

for index, row in goes_df.iterrows():
    if str(row['19.8']).startswith('+'):
        start_time = pd.to_datetime(row['Start']) #zmena stĺpca label podľa dát navrátených zo Srbska
        end_time = pd.to_datetime(row['End'])
        
        khz198.loc[(khz198['time'] >= start_time) & 
                   (khz198['time'] <= end_time), 'label'] = 1

In [10]:
khz198[khz198['label']!=0]

,time,19.8kHz,label
152154,2013-05-20 14:59:00,46.6471,1
152155,2013-05-20 15:00:00,46.0478,1
152156,2013-05-20 15:01:00,46.4566,1
152157,2013-05-20 15:02:00,46.4605,1
152158,2013-05-20 15:03:00,46.7686,1
...,...,...,...
326975,2013-11-19 10:30:00,39.7496,1
326976,2013-11-19 10:31:00,38.4399,1
326977,2013-11-19 10:32:00,39.3626,1
326978,2013-11-19 10:33:00,39.2509,1


In [11]:
goes_df = pd.read_csv('goes_xray_event_list_2013_Serbia.csv')

khz2027['time'] = pd.to_datetime(khz2027['time'])

for index, row in goes_df.iterrows():
    if str(row['20.27']).startswith('+'):
        start_time = pd.to_datetime(row['Start']) #zmena stĺpca label podľa dát navrátených zo Srbska
        end_time = pd.to_datetime(row['End'])
        
        khz2027.loc[(khz2027['time'] >= start_time) & 
                   (khz2027['time'] <= end_time), 'label'] = 1

In [12]:
khz2027[khz2027['label']!=0]

,time,20.27kHz,label
65345,2013-02-20 11:08:00,67.2019,1
65346,2013-02-20 11:09:00,67.1471,1
65347,2013-02-20 11:10:00,66.9140,1
65348,2013-02-20 11:11:00,69.5970,1
65349,2013-02-20 11:12:00,71.7408,1
...,...,...,...
332851,2013-11-23 12:34:00,55.9691,1
332852,2013-11-23 12:35:00,56.7289,1
332853,2013-11-23 12:36:00,55.6781,1
332854,2013-11-23 12:37:00,55.6746,1


In [13]:
goes_df = pd.read_csv('goes_xray_event_list_2013_Serbia.csv')

khz234['time'] = pd.to_datetime(khz234['time'])

for index, row in goes_df.iterrows():
    if str(row['19.8']).startswith('+'):
        start_time = pd.to_datetime(row['Start'])
        end_time = pd.to_datetime(row['End']) #zmena stĺpca label podľa dát navrátených zo Srbska
        
        khz234.loc[(khz234['time'] >= start_time) & 
                   (khz234['time'] <= end_time), 'label'] = 1

In [14]:
khz234[khz234['label']!=0]

,time,23.4kHz,label
152154,2013-05-20 14:59:00,78.0636,1
152155,2013-05-20 15:00:00,78.0229,1
152156,2013-05-20 15:01:00,77.9881,1
152157,2013-05-20 15:02:00,77.9784,1
152158,2013-05-20 15:03:00,78.1038,1
...,...,...,...
326975,2013-11-19 10:30:00,71.3163,1
326976,2013-11-19 10:31:00,70.6423,1
326977,2013-11-19 10:32:00,70.8277,1
326978,2013-11-19 10:33:00,71.0841,1


In [15]:
goes_df = pd.read_csv('goes_xray_event_list_2013_Serbia.csv')

khz221['time'] = pd.to_datetime(khz221['time'])

for index, row in goes_df.iterrows():
    if str(row['22.1']).startswith('+'):
        start_time = pd.to_datetime(row['Start']) #zmena stĺpca label podľa dát navrátených zo Srbska
        end_time = pd.to_datetime(row['End'])
        
        khz221.loc[(khz221['time'] >= start_time) & 
                   (khz221['time'] <= end_time), 'label'] = 1

In [16]:
khz221[khz221['label']!=0]

,time,22.1kHz,label
65345,2013-02-20 11:08:00,65.2029,1
65346,2013-02-20 11:09:00,65.1271,1
65347,2013-02-20 11:10:00,64.6846,1
65348,2013-02-20 11:11:00,62.7888,1
65349,2013-02-20 11:12:00,61.3283,1
...,...,...,...
332851,2013-11-23 12:34:00,51.6024,1
332852,2013-11-23 12:35:00,52.5949,1
332853,2013-11-23 12:36:00,51.7464,1
332854,2013-11-23 12:37:00,52.0200,1


In [18]:
goes_df = pd.read_csv('goes_xray_event_list_2013_Serbia.csv')

khz24['time'] = pd.to_datetime(khz24['time'])

for index, row in goes_df.iterrows():
    if str(row['24']).startswith('+'):
        start_time = pd.to_datetime(row['Start']) #zmena stĺpca label podľa dát navrátených zo Srbska
        end_time = pd.to_datetime(row['End'])
        
        khz24.loc[(khz24['time'] >= start_time) & 
                   (khz24['time'] <= end_time), 'label'] = 1

In [19]:
khz24[khz24['label']!=0]

,time,24kHz,label
8006,2013-01-11 14:51:00,44.3739,1
8007,2013-01-11 14:52:00,44.2178,1
8008,2013-01-11 14:53:00,44.2159,1
8009,2013-01-11 14:54:00,44.2315,1
8010,2013-01-11 14:55:00,44.4830,1
...,...,...,...
332878,2013-11-23 13:01:00,45.4161,1
332879,2013-11-23 13:02:00,44.9921,1
332880,2013-11-23 13:03:00,45.0428,1
332881,2013-11-23 13:04:00,45.0066,1


In [20]:
khz2027.to_csv('fraqdata/khz2027.csv', index=False)
khz198.to_csv('fraqdata/khz198.csv', index=False)
khz234.to_csv('fraqdata/khz234.csv', index=False)  #uloženie datasetov
khz221.to_csv('fraqdata/khz221.csv', index=False)
khz24.to_csv('fraqdata/khz24.csv', index=False)